In [1]:
import os
import re
import pandas as pd
import importlib

## Extracción

In [36]:
import FuncionesLectura as fl
importlib.reload(fl)

df_corpus=fl.leerCorpus(hacerLimpieza=True)
print(df_corpus.shape)
df_corpus.sample(5)

Antes de limpiar: (6471, 9)
Después de eliminar español: (6098, 9)
Después de eliminar vacíos: (6097, 9)
Después de limpiar duplicados: (5619, 9)
(5617, 9)


,id,speaker,transcription,text,morpheme_break,pos,gloss_es,free_translation,file
3420,JP-cuento.colorado-2013_392,José,tapas menara ha honi tsamake ironki hara tapas,tapas menara ha honi tsamake ...,tapas mená =ra ha honi tsamat ...,n. pos. =clit. dem n. v. ...,guarida LOC =EVID1(SK) ese(SK) hombre(SK) re...,dentro de ese tambito la gente dicen que entra...,JP-cuento.colorado-2013.txt
3872,NC-narracion.vida2-2013_032,Nelita,aranares,aranares,None,None,None,eso es todo,NC-narracion.vida2-2013.txt
5668,DICCIONARIOISKONAWA7_912,Dictionary,mehko,None,None,n.,pez lobo o huasaco,"pez grande, de cuerpo alargado y grueso.",DICCIONARIOISKONAWA7.txt
5335,DICCIONARIOISKONAWA7_579,Dictionary,imi iwi,None,None,n.,sangre de grado o sangre de grado,"Árbol alto, de tronco blanco y grueso, hojas r...",DICCIONARIOISKONAWA7.txt
4488,PC-cuento.beha.tao-2013.eaf_a165,Pablo,bishkoa richikesa richi richibika,bishkoa richikesa richi richibika,None,None,None,"rayado (bota) como leche, como leche misma",PC-cuento.beha.tao-2013.eaf


In [ ]:
df_corpus.sample(5)

In [37]:
df_corpus.to_json('corpus.json', orient='records', lines=True)

## Prueba de lectura

In [3]:
df_corpus_leido = pd.read_json('corpus.json', lines=True)
df_corpus_leido.sample(5)

,id,speaker,transcription,text,morpheme_break,pos,gloss_es,free_translation,file
2400,JP-cuento.animal-2013_125,jose,hae,hae,hae,n.,oso.hormiguero,Oso hormiguero.,JP-cuento.animal-2013.txt
1796,JC.NC.IC-paucar.del.mani-2013_050,Juanita,kankan,kankan,kankan,n.,canasta,canasta,JC.NC.IC-paucar.del.mani-2013.txt
2566,JP-cuento.colorado-2013_123,José,miara oni bakebo mia pinahani iki miara pinahani,miara oni bakebo mia ...,mia =ra oni bake =bo mia ...,pron =clit. n. n. =clit pron...,2SG.ABS(SK) =EVID1(SK) persona hijo =PLU 2SG.A...,los hijos de los hombres te van a comer en cua...,JP-cuento.colorado-2013.txt
1812,JC.NC.IC-paucar.del.mani-2013_072,Juanita,karestimari epá,karestimari epá,ka -res -ti =ma =ri epá,v.intr -suf. -suf. =clit. =clit n.,ir -solamente -NOMLZ =NEG =COP.INT papá.VOC,"""vamos a caminar no más, papá""",JC.NC.IC-paucar.del.mani-2013.txt
1300,JC.NC.IC-conversacion.varios-2013_236,Nelita,Miaribi awitsahakonahain,Miaribi awitsahakonahain,None,None,None,Tú también [?¿],JC.NC.IC-conversacion.varios-2013.txt


## Validaciones

In [7]:
#Verificar que las columnas id,	speaker,	transcription, free_translation, file no tengan valores nulos
assert df_corpus_leido.id.isnull().sum() == 0
assert df_corpus_leido.speaker.isnull().sum() == 0  
assert df_corpus_leido.transcription.isnull().sum() == 0
assert df_corpus_leido.free_translation.isnull().sum() == 0
assert df_corpus_leido.file.isnull().sum() == 0

print('No hay valores nulos en el corpus')

No hay valores nulos en el corpus
